In [12]:
import sys
sys.path.insert(0, '../FeatureExtract')
from generating_feature import generating_pro_feature, generating_drug_feature, concat_feature
import pandas as pd
import numpy as np

In [13]:
# import pickle


# dind = np.zeros(1710)
# pind = np.zeros(34123)
# with open('../data/{}/DAVIS_drug_feats.pkl'.format("davis"), 'rb') as f:
#     drug = pickle.load(f, encoding='utf-8')
# with open('../data/{}/DAVIS_target_feats_tmp.pkl'.format("davis"), 'rb') as f:
#     pro = pickle.load(f, encoding='utf-8')

# print(drug.keys())
# print(pro.keys())
# # print(drug['Drug_ID'])
# # print(len(drugfeat['Morgan_Features']))

# input = np.array([1549, 1240, 764])
# indices = np.where(np.array(drug['Drug_enco']) == input[:, None])[1]
# res = pd.DataFrame(drug['Morgan_Features'].iloc[indices])
# print(type(res))
# print(res)

# # drugfeat['Morgan_Features'],   drugfeat['Drug_ID']
# # profeat['ProtBERT_Features'],  profeat['Target_ID']

In [14]:
import pickle
from sklearn import svm
from sklearn.metrics import accuracy_score, f1_score

def my_SVM(df_train, df_val, df_test):
    # df_train : Target, Drug, Y
    with open('../data/{}/DAVIS_target_feats_tmp.pkl'.format("davis"), 'rb') as f:
        pro = pickle.load(f, encoding='utf-8')
    proenco = pro['Gene_enco']
    profeat = pro['ProtBERT_Features']

    with open('../data/{}/DAVIS_drug_feats.pkl'.format("davis"), 'rb') as f:
        drug = pickle.load(f, encoding='utf-8')
    drugenco = drug['Drug_enco']
    drugfeat = drug['Morgan_Features']

    def get_features(indices, feat):
        feat_df = pd.DataFrame(feat.iloc[indices].tolist())
        feat_df = feat_df.reset_index(drop=True)
        return feat_df

    # get features
    train_d_indices = np.where(np.array(drugenco) == np.array(df_train[1])[:, None])[1]
    train_p_indices = np.where(np.array(proenco) == np.array(df_train[0])[:, None])[1]
    train_d_feat = get_features(train_d_indices, drugfeat)
    train_p_feat = get_features(train_p_indices, profeat)

    val_d_indices = np.where(np.array(drugenco) == np.array(df_val[1])[:, None])[1]
    val_p_indices = np.where(np.array(proenco) == np.array(df_val[0])[:, None])[1]
    val_d_feat = get_features(val_d_indices, drugfeat)
    val_p_feat = get_features(val_p_indices, profeat)

    test_d_indices = np.where(np.array(drugenco) == np.array(df_test[1])[:, None])[1]
    test_p_indices = np.where(np.array(proenco) == np.array(df_test[0])[:, None])[1]
    test_d_feat = get_features(test_d_indices, drugfeat)
    test_p_feat = get_features(test_p_indices, profeat)

    clf = svm.SVC(kernel='rbf')
    clf.fit(pd.concat([train_p_feat, train_d_feat], axis=1), df_train[2])
    
    # Evaluation
    pred_rels_train = clf.predict(pd.concat([train_p_feat, train_d_feat], axis=1))
    accuracy_train = accuracy_score(df_train[2], pred_rels_train)
    f1score_train = f1_score(df_train[2], pred_rels_train)

    pred_rels_val = clf.predict(pd.concat([val_p_feat, val_d_feat], axis=1))
    accuracy_val = accuracy_score(df_val[2], pred_rels_val)
    f1score_val = f1_score(df_val[2], pred_rels_val)
    
    pred_rels_test = clf.predict(pd.concat([test_p_feat, test_d_feat], axis=1))
    accuracy_test = accuracy_score(df_test[2], pred_rels_test)
    f1score_test = f1_score(df_test[2], pred_rels_test)

    print("Train -> ", "Accuracy:", round(accuracy_train, 4), "F1-score", round(f1score_train, 4))
    print("Validation -> ","Accuracy:", round(accuracy_val, 4), "F1-score", round(f1score_val, 4))
    print("Test -> ","Accuracy:", round(accuracy_test, 4), "F1-score", round(f1score_test, 4))

df_train = pd.read_csv("../data/davis/train.txt", sep=' ',header=None)
df_val = pd.read_csv("../data/davis/dev.txt", sep=' ', header=None)
df_test = pd.read_csv("../data/davis/test.txt", sep=' ',header=None)

my_SVM(df_train, df_val, df_test)

Train ->  Accuracy: 0.9596 F1-score 0.0
Validation ->  Accuracy: 0.9749 F1-score 0.0
Test ->  Accuracy: 0.967 F1-score 0.0


In [15]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score

def my_XGBoost(df_train, df_val, df_test):
    def prepare_data(df):
       # df: Target, Drug, Y
        with open('../data/{}/DAVIS_target_feats_tmp.pkl'.format("davis"), 'rb') as f:
            pro = pickle.load(f, encoding='utf-8')
        proenco = pro['Gene_enco']
        profeat = pro['ProtBERT_Features']

        with open('../data/{}/DAVIS_drug_feats.pkl'.format("davis"), 'rb') as f:
            drug = pickle.load(f, encoding='utf-8')
        drugenco = drug['Drug_enco']
        drugfeat = drug['Morgan_Features']

        def get_features(indices, feat):
            feat_df = pd.DataFrame(feat.iloc[indices].tolist())
            feat_df = feat_df.reset_index(drop=True)
            return feat_df

        # get features
        d_indices = np.where(np.array(drugenco) == np.array(df[1])[:, None])[1]
        p_indices = np.where(np.array(proenco) == np.array(df[0])[:, None])[1]
        d_feat = get_features(d_indices, drugfeat)
        p_feat = get_features(p_indices, profeat)

        
        concat_feats = pd.concat([p_feat, d_feat], axis=1)
        gt_rels = df[2]
        
        concat_feats.columns = range(concat_feats.shape[1])
        
        return xgb.DMatrix(concat_feats, label=gt_rels)
    
    dtrain = prepare_data(df_train)
    dval = prepare_data(df_val)
    dtest = prepare_data(df_test)

    params = {
        'objective': 'binary:logistic',
        'eval_metric': ['error', 'logloss', 'auc'],
        'max_depth': 3,
        'learning_rate': 0.1
    }
    
    model = xgb.train(params, dtrain)
    
    def evaluate(dmatrix, gt_rels):
        pred = model.predict(dmatrix)
        pred_binary = [1 if p >= 0.5 else 0 for p in pred]
        accuracy = accuracy_score(gt_rels, pred_binary)
        f1score = f1_score(gt_rels, pred_binary)
        return accuracy, f1score
    
    accuracy_train, f1score_train = evaluate(dtrain, df_train[2])
    accuracy_val, f1score_val = evaluate(dval, df_val[2])
    accuracy_test, f1score_test = evaluate(dtest, df_test[2])
    
    print("Train -> ", "Accuracy:", round(accuracy_train, 4), "F1-score", round(f1score_train, 4))
    print("Validation -> ","Accuracy:", round(accuracy_val, 4), "F1-score", round(f1score_val, 4))
    print("Test -> ","Accuracy:", round(accuracy_test, 4), "F1-score", round(f1score_test, 4))


df_train = pd.read_csv("../data/davis/train.txt", sep=' ',header=None)
df_val = pd.read_csv("../data/davis/dev.txt", sep=' ', header=None)
df_test = pd.read_csv("../data/davis/test.txt", sep=' ',header=None)

my_XGBoost(df_train, df_val, df_test)


Train ->  Accuracy: 0.9596 F1-score 0.0
Validation ->  Accuracy: 0.9749 F1-score 0.0
Test ->  Accuracy: 0.967 F1-score 0.0


In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

def my_RandomForest(df_train, df_val, df_test):
    def prepare_data(df):
        with open('../data/{}/DAVIS_target_feats_tmp.pkl'.format("davis"), 'rb') as f:
            pro = pickle.load(f, encoding='utf-8')
        proenco = pro['Gene_enco']
        profeat = pro['ProtBERT_Features']

        with open('../data/{}/DAVIS_drug_feats.pkl'.format("davis"), 'rb') as f:
            drug = pickle.load(f, encoding='utf-8')
        drugenco = drug['Drug_enco']
        drugfeat = drug['Morgan_Features']

        def get_features(indices, feat):
            feat_df = pd.DataFrame(feat.iloc[indices].tolist())
            feat_df = feat_df.reset_index(drop=True)
            return feat_df

        # Get features
        d_indices = np.where(np.array(drugenco) == np.array(df[1])[:, None])[1]
        p_indices = np.where(np.array(proenco) == np.array(df[0])[:, None])[1]
        d_feat = get_features(d_indices, drugfeat)
        p_feat = get_features(p_indices, profeat)

        concat_feats = pd.concat([p_feat, d_feat], axis=1)
        gt_rels = df[2]

        return concat_feats, gt_rels

    train_feats, train_gt_rels = prepare_data(df_train)
    val_feats, val_gt_rels = prepare_data(df_val)
    test_feats, test_gt_rels = prepare_data(df_test)

    clf = RandomForestClassifier()
    clf.fit(train_feats, train_gt_rels)

    def evaluate(feats, gt_rels):
        pred_rels = clf.predict(feats)
        accuracy = accuracy_score(gt_rels, pred_rels)
        f1score = f1_score(gt_rels, pred_rels)
        return accuracy, f1score

    accuracy_train, f1score_train = evaluate(train_feats, train_gt_rels)
    accuracy_val, f1score_val = evaluate(val_feats, val_gt_rels)
    accuracy_test, f1score_test = evaluate(test_feats, test_gt_rels)

    print("Train -> ", "Accuracy:", round(accuracy_train, 4), "F1-score", round(f1score_train, 4))
    print("Validation -> ","Accuracy:", round(accuracy_val, 4), "F1-score", round(f1score_val, 4))
    print("Test -> ","Accuracy:", round(accuracy_test, 4), "F1-score", round(f1score_test, 4))


df_train = pd.read_csv("../data/davis/train.txt", sep=' ', header=None)
df_val = pd.read_csv("../data/davis/dev.txt", sep=' ', header=None)
df_test = pd.read_csv("../data/davis/test.txt", sep=' ', header=None)

my_RandomForest(df_train, df_val, df_test)

Train ->  Accuracy: 0.9596 F1-score 0.0
Validation ->  Accuracy: 0.9749 F1-score 0.0
Test ->  Accuracy: 0.967 F1-score 0.0


In [18]:
df_train = pd.read_csv("../data/davis/train.txt", sep=' ', header=None)
df_val = pd.read_csv("../data/davis/dev.txt", sep=' ', header=None)
df_test = pd.read_csv("../data/davis/test.txt", sep=' ', header=None)

print("SVM model Result")
my_SVM(df_train, df_val, df_test)
print()
print("XGBoost model Result")
my_XGBoost(df_train, df_val, df_test)
print()
print("RandomForest model Result")
my_RandomForest(df_train, df_val, df_test)

SVM model Result
Train ->  Accuracy: 0.9596 F1-score 0.0
Validation ->  Accuracy: 0.9749 F1-score 0.0
Test ->  Accuracy: 0.967 F1-score 0.0

XGBoost model Result
Train ->  Accuracy: 0.9596 F1-score 0.0
Validation ->  Accuracy: 0.9749 F1-score 0.0
Test ->  Accuracy: 0.967 F1-score 0.0

RandomForest model Result
Train ->  Accuracy: 0.9596 F1-score 0.0
Validation ->  Accuracy: 0.9749 F1-score 0.0
Test ->  Accuracy: 0.967 F1-score 0.0
